In [32]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Define the folder containing the CSV files
result_folder = 'resutlados eval graf\\resultados para evaluar sac gbpusd m'

# Iterate through the CSV files in the folder
for filename in os.listdir(result_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(result_folder, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Plot the values of Loss, Value Loss, Policy Loss, and Entropy as a function of Step
        plt.figure(figsize=(10, 6))
        
        plt.plot(df['Step'], df['Actor Loss'], label='Actor Loss')
        plt.plot(df['Step'], df['Critic Loss'], label='Critic Loss')
        plt.plot(df['Step'], df['Entropy Coef'], label='Entropy Coef')
        plt.plot(df['Step'], df['Entropy Coef Loss'], label='Entropy Coef Loss')
        
        plt.xlabel('Step')
        plt.ylabel('Value')
        plt.title(f'Training Statistics for {filename}')
        plt.legend()
        
        
        # Save the plot as a PNG file
        plot_filename = os.path.splitext(filename)[0] + '_plot.png'
        plot_path = os.path.join(result_folder, plot_filename)
        plt.savefig(plot_path)
        
        # Close the plot to free up memory
        plt.close()

print('Plots saved in the result_ent folder.')

Plots saved in the result_ent folder.
